<a href="https://colab.research.google.com/github/wandieinnocents/Sales_analysis_linearRegression-XGboost-Randomforest/blob/main/sales_analysis_linearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,  LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [23]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [24]:
# read the dataset 
store_sales = pd.read_csv("/content/drive/MyDrive/research/project2/dataset/train.csv")
store_sales.head(10)

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
5,2013-01-06,1,1,12
6,2013-01-07,1,1,10
7,2013-01-08,1,1,9
8,2013-01-09,1,1,12
9,2013-01-10,1,1,9


Check for null values

In [25]:
# check the dataset to know it 
store_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    913000 non-null  object
 1   store   913000 non-null  int64 
 2   item    913000 non-null  int64 
 3   sales   913000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 27.9+ MB


There are no null values in the dataset

Since am looking at getting the overal sales for all the stores , i drop the item , and storeID

In [26]:
# drop the store and item
store_sales = store_sales.drop(['store','item'], axis=1)
store_sales

,date,sales
0,2013-01-01,13
1,2013-01-02,11
2,2013-01-03,14
3,2013-01-04,13
4,2013-01-05,10
...,...,...
912995,2017-12-27,63
912996,2017-12-28,59
912997,2017-12-29,74
912998,2017-12-30,62


In [27]:
# get infor of the data
store_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    913000 non-null  object
 1   sales   913000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 13.9+ MB


##Converting date from object datatype to datetime datatype 

The date is of object datatype, which is not correct , i convert it to datetime datatype

In [28]:
# converting date
store_sales['date'] = pd.to_datetime(store_sales['date'])

In [29]:
store_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    913000 non-null  datetime64[ns]
 1   sales   913000 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 13.9 MB


#WE WANT TO TRAIN THE MODEL TO PREDICT SALES IN NEXT MONTH INSTEAD OF DAY
- I convert the date to the period of a month
- Then i add/summ-up all the items sold in each month


In [30]:
# convert date to period of month and sum all items sold in each month
# dt.to_period allows for converting a date to a specific interval.
store_sales['date'] = store_sales['date'].dt.to_period("M")
# montly sales 
monthly_sales = store_sales.groupby('date').sum().reset_index()

Converting the date column to timestamp datatype

In [31]:
monthly_sales['date'] = monthly_sales['date'].dt.to_timestamp()

In [34]:
monthly_sales.head(10)

,date,sales
0,2013-01-01,454904
1,2013-02-01,459417
2,2013-03-01,617382
3,2013-04-01,682274
4,2013-05-01,763242
5,2013-06-01,795597
6,2013-07-01,855922
7,2013-08-01,766761
8,2013-09-01,689907
9,2013-10-01,656587
